In [1]:
import pandas as pd
import numpy as np


# Loading the training data

Loading the list of foods and description/ingredients list from the USDA (Source: https://fdc.nal.usda.gov/download-datasets.html)

In [2]:
branded_food_cols = ["fdc_id","brand_owner","brand_name","ingredients","not_a_significant_source_of","branded_food_category"]

In [8]:
df_branded_food = pd.read_csv("branded_food.csv", usecols = branded_food_cols, on_bad_lines = 'skip')

<ipython-input-8-82944e97e487>:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df_branded_food = pd.read_csv("branded_food.csv", usecols = branded_food_cols, on_bad_lines = 'skip')


In [9]:
df_branded_food.head()

,fdc_id,brand_owner,brand_name,ingredients,not_a_significant_source_of,branded_food_category
0,1105904,Richardson Oilseed Products (US) Limited,NaN,Vegetable Oil,NaN,Oils Edible
1,1105905,CAMPBELL SOUP COMPANY,NaN,"INGREDIENTS: BEEF STOCK, CONTAINS LESS THAN 2%...",NaN,Herbs/Spices/Extracts
2,1105906,CAMPBELL SOUP COMPANY,NaN,"INGREDIENTS: CLAM STOCK, POTATOES, CLAMS, CREA...",NaN,Prepared Soups
3,1105907,CAMPBELL SOUP COMPANY,NaN,"INGREDIENTS: WATER, CREAM, BROCCOLI, CELERY, V...",NaN,Prepared Soups
4,1105908,CAMPBELL SOUP COMPANY,NaN,"INGREDIENTS: CHICKEN STOCK, CONTAINS LESS THAN...",NaN,Herbs/Spices/Extracts


In [10]:
df_branded_food.shape

(196802, 6)

Loading the list of foods and their names

# Feature Engineering

## Flagging ingredients

In [11]:
food_ingredients = df_branded_food['ingredients']

In [12]:
word_to_search = 'Gluten'

# Create a new column to flag rows containing the word
df_branded_food['is_gluten'] = food_ingredients.str.contains(word_to_search, case=False).astype(bool)


In [13]:
df_branded_food['is_gluten'].value_counts()

False    186521
True      10281
Name: is_gluten, dtype: int64

In [14]:
meat_list= ["Beef", "Pork", "Chicken", "Lamb", "Turkey", "Duck", "Goose", "Veal", "Venison", "Bison", "Rabbit", "Sausage", "Salami", "Ham", "Bacon", "Prosciutto", "Pancetta", "Chorizo", "Pepperoni", "Jerky", "Schnitzel", "Steak", "Rib", "Loin", "Chop", "Breast", "Thigh", "Wing", "Leg", "Drumstick", "Shank", "processed meats", "horsemeat", "poultry", "deer", "deli", "fowl", "pigmeat", "cow", "turkeys", "cows", "gravy", "cattle", "snail", "crocodile", "sheep", "sheepmeat", "sirloin"]

In [15]:
# Should probably split this out for pork / beef because of religious dietary restrictions

words_to_search = meat_list

pattern = '|'.join(words_to_search)

df_branded_food['contains_meat'] = food_ingredients.str.contains(pattern, case=False).astype(bool)

In [16]:
df_branded_food['contains_meat'].value_counts()

False    141011
True      55791
Name: contains_meat, dtype: int64

In [17]:
shellfish_food = ["Shrimp", "Crab", "Lobster", "Clam", "Mussel", "Oyster", "Scallop", "Abalone", "Squid", "Octopus", "Cuttlefish", "Snail", "Conch", "Periwinkle", "Whelk", "Cockle", "Razor clam", "Geoduck", "Barnacle", "Sea urchin", "Sea cucumber", "Limpet", "Winkle", "Krill", "Prawn", "Langoustine", "Crawfish", "Crayfish", "Slipper lobster", "Horseshoe crab", "Mantis shrimp", "Sand dollar", "Sea snail", "Sea slug", "Sea squirt", "Trepang (sea cucumber)", "Chiton", "uni", "escargot", "crabmeat",]

In [18]:
words_to_search = shellfish_food

pattern = '|'.join(words_to_search)

df_branded_food['contains_shellfish'] = food_ingredients.str.contains(pattern, case=False).astype(bool)

<ipython-input-18-3162bfa3cb0c>:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_branded_food['contains_shellfish'] = food_ingredients.str.contains(pattern, case=False).astype(bool)


In [19]:
df_branded_food['contains_shellfish'].value_counts()

False    192974
True       3828
Name: contains_shellfish, dtype: int64

In [20]:
nuts_list = ["Almond", "Walnut", "Cashew", "Pistachio", "Pecan", "Hazelnut", "Brazil nut", "Macadamia nut", "Pine nut", "Chestnut", "Peanut"]

In [21]:
words_to_search = nuts_list

pattern = '|'.join(words_to_search)

df_branded_food['contains_nuts'] = food_ingredients.str.contains(pattern, case=False).astype(bool)

In [22]:
df_branded_food['contains_nuts'].value_counts()

False    175588
True      21214
Name: contains_nuts, dtype: int64

In [23]:
diary_list = ["Milk", "Diary", "Butter", "Cheese", "Yogurt", "Cream", "Sour cream", "Buttermilk", "Whey", "Casein", "Caseinate", "Curds", "Ghee", "Lactose", "Milkfat", "Half-and-half", "Cottage cheese", "Ricotta cheese", "Cream cheese", "Mascarpone", "Ice cream", "Frozen yogurt", "Custard", "Pudding", "Cheddar", "Mozzarella", "Parmesan", "Swiss", "Gouda", "Brie", "Feta", "Blue cheese", "Provolone", "Monterey Jack", "Colby", "Pepper Jack", "Ricotta", "Havarti", "Muenster", "Camembert", "Fontina", "Gruyere", "Edam", "Asiago", "Manchego", "Cotija", "Queso fresco", "Queso blanco", "Panela", "Halloumi", "Teleme", "Burrata", "Burrini", "Cottage cheese", "Cream cheese", "Neufchâtel", "Quark", "Fromage blanc", "Chevre (goat cheese)", "Roquefort", "Stilton", "Cambozola", "Gorgonzola", "Taleggio", "Emmental", "Limburger", "Taleggio", "Pecorino Romano", "Pecorino Toscano", "Manouri", "Myzithra", "Kasseri", "Kefalotyri", "Mizithra", "Parmigiano-Reggiano", "Grana Padano", "Ricotta salata", "Saganaki", "Sirene", "Tulum", "Tzfat", "Västerbottenost", "Wensleydale"]

In [24]:
words_to_search = diary_list

pattern = '|'.join(words_to_search)

df_branded_food['contains_diary'] = food_ingredients.str.contains(pattern, case=False).astype(bool)

<ipython-input-24-64c7f7767a92>:5: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_branded_food['contains_diary'] = food_ingredients.str.contains(pattern, case=False).astype(bool)


In [25]:
df_branded_food['contains_diary'].value_counts()

False    124098
True      72704
Name: contains_diary, dtype: int64

In [26]:
soy_list = ["Soybeans", "Tofu", "Tempeh", "Edamame", "Soy milk", "Soy sauce", "Miso", "Natto", "Soy flour", "soy", "bean curd", "tamari", "shoyu", "Soybean", "Textured vegetable protein"]

In [27]:
words_to_search = soy_list

pattern = '|'.join(words_to_search)

df_branded_food['contains_soy'] = food_ingredients.str.contains(pattern, case=False).astype(bool)

In [28]:
egg_list = ["Egg","Eggshell","Mayonnaise","Mayo","Bearnaise","caesar","custard","pudding","Quiche","Soufflé","Meringue","Frittata","Omelette"]

In [29]:
words_to_search = egg_list

pattern = '|'.join(words_to_search)

df_branded_food['contains_egg'] = food_ingredients.str.contains(pattern, case=False).astype(bool)

In [30]:
animal_byproducts = ["Gelatin", "Tallow", "Lard", "Rennet", "Isinglass", "Fish oil", "Bone char", "Carmine", "Shellac", "Collagen", "Beeswax", "Lanolin", "lipase", "protease", "amylase", "lecithin", "Stearic acid ", "Oleic acid ", "Glycerides", "diglycerides", "monoglycerides", "suet", "Monostearate", "Polysorbates", "carmine", "Cochineal extract", "Natural red 4", "E120", "Natural red coloring", "Crimson lake", "Red food colorant"]

In [31]:
# This is a really hard list to nail since things like vitamins don't specify animal source so they could be anti-vegan

words_to_search = animal_byproducts

pattern = '|'.join(words_to_search)

df_branded_food['contains_animal_byproducts'] = food_ingredients.str.contains(pattern, case=False).astype(bool)

## Categorizing by diet

In [32]:
# Vegan

df_branded_food["is_vegan"] = ~(df_branded_food['contains_meat'] \
                                | df_branded_food['contains_shellfish'] \
                                | df_branded_food['contains_diary'] \
                                | df_branded_food['contains_egg'] \
                                | df_branded_food['contains_animal_byproducts'])

In [33]:
# Vegetarian

df_branded_food["is_vegetarian"] = ~(df_branded_food['contains_meat'] \
                                | df_branded_food['contains_shellfish'])


In [34]:
# Pescatarian

df_branded_food["is_pescatarian"] = ~df_branded_food['contains_meat']

In [35]:
df_branded_food.head()

,fdc_id,brand_owner,brand_name,ingredients,not_a_significant_source_of,branded_food_category,is_gluten,contains_meat,contains_shellfish,contains_nuts,contains_diary,contains_soy,contains_egg,contains_animal_byproducts,is_vegan,is_vegetarian,is_pescatarian
0,1105904,Richardson Oilseed Products (US) Limited,NaN,Vegetable Oil,NaN,Oils Edible,False,False,False,False,False,False,False,False,True,True,True
1,1105905,CAMPBELL SOUP COMPANY,NaN,"INGREDIENTS: BEEF STOCK, CONTAINS LESS THAN 2%...",NaN,Herbs/Spices/Extracts,False,True,False,False,False,False,False,False,False,False,False
2,1105906,CAMPBELL SOUP COMPANY,NaN,"INGREDIENTS: CLAM STOCK, POTATOES, CLAMS, CREA...",NaN,Prepared Soups,False,True,True,False,True,True,False,True,False,False,False
3,1105907,CAMPBELL SOUP COMPANY,NaN,"INGREDIENTS: WATER, CREAM, BROCCOLI, CELERY, V...",NaN,Prepared Soups,False,False,False,False,True,True,False,True,False,True,True
4,1105908,CAMPBELL SOUP COMPANY,NaN,"INGREDIENTS: CHICKEN STOCK, CONTAINS LESS THAN...",NaN,Herbs/Spices/Extracts,False,True,False,False,False,False,False,False,False,False,False


# Writing to a new csv

In [36]:
df_branded_food.to_csv('data.csv', index=False)